In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.drop2 = nn.BatchNorm1d(embedding_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        embedded = self.drop2(embedded)
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        self.drop2 = nn.BatchNorm1d(embedding_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        embedded = self.drop2(embedded)
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         file1 = open("loss_steps.txt", "a")  # append mode
#         file1.write(str(loss.item()) + ",")
#         file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))
    

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 6

for epoch in range(1, num_epoch + 1):
    train(encoder, decoder, device, train_all_loader, encoder_optimizer, decoder_optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
26725,"[410.64337158203125, 1154.22998046875]","[411.249267578125, 1159.4306640625]","[412.5081481933594, 1161.099853515625]","[413.1223449707031, 1162.5858154296875]","[413.6150207519531, 1163.8905029296875]","[414.02667236328125, 1165.08837890625]","[414.39019775390625, 1166.20458984375]","[414.7232666015625, 1167.253662109375]","[415.03277587890625, 1168.249755859375]","[415.3131408691406, 1169.211181640625]",...,"[416.59808349609375, 1179.7733154296875]","[416.7009582519531, 1180.7354736328125]","[416.8116149902344, 1181.6962890625]","[416.9306945800781, 1182.6551513671875]","[417.05865478515625, 1183.61181640625]","[417.19573974609375, 1184.5648193359375]","[417.3417053222656, 1185.5120849609375]","[417.4955749511719, 1186.4495849609375]","[417.65533447265625, 1187.3712158203125]","[417.8173828125, 1188.2677001953125]"
10945,"[1729.3343505859375, 465.8460998535156]","[1732.1790771484375, 468.02239990234375]","[1729.8519287109375, 470.93450927734375]","[1729.05224609375, 471.77288818359375]","[1728.3192138671875, 472.521484375]","[1727.731201171875, 473.1347961425781]","[1727.2392578125, 473.69091796875]","[1726.8179931640625, 474.2107849121094]","[1726.4498291015625, 474.7023620605469]","[1726.124755859375, 475.1698303222656]",...,"[1724.25146484375, 479.5301513671875]","[1724.200439453125, 479.9322509765625]","[1724.1702880859375, 480.3522033691406]","[1724.1629638671875, 480.7959899902344]","[1724.181396484375, 481.27093505859375]","[1724.229248046875, 481.7862243652344]","[1724.31201171875, 482.3535461425781]","[1724.437255859375, 482.9881896972656]","[1724.63671875, 483.629638671875]","[1724.857421875, 484.3134460449219]"
583,"[731.9131469726562, 1549.9996337890625]","[732.0345458984375, 1545.67578125]","[732.6150512695312, 1544.5521240234375]","[733.0922241210938, 1543.130615234375]","[733.5868530273438, 1542.1285400390625]","[734.0125732421875, 1541.110595703125]","[734.3973999023438, 1540.1707763671875]","[734.740234375, 1539.2628173828125]","[735.0501098632812, 1538.400634765625]","[735.3302612304688, 1537.5782470703125]",...,"[737.1251831054688, 1530.82568359375]","[737.205810546875, 1530.3746337890625]","[737.2763671875, 1529.9439697265625]","[737.3372192382812, 1529.53271484375]","[737.388916015625, 1529.1396484375]","[737.431884765625, 1528.7637939453125]","[737.4664916992188, 1528.404296875]","[737.4929809570312, 1528.0601806640625]","[737.5117797851562, 1527.730712890625]","[737.5230102539062, 1527.415283203125]"
37070,"[189.3434295654297, 1025.1285400390625]","[193.45919799804688, 1026.066162109375]","[196.35989379882812, 1026.4276123046875]","[198.421142578125, 1026.697265625]","[200.27099609375, 1026.8572998046875]","[201.94308471679688, 1027.0440673828125]","[203.47462463378906, 1027.26171875]","[204.8808135986328, 1027.50341796875]","[206.1669921875, 1027.760009765625]","[207.33396911621094, 1028.0218505859375]",...,"[213.47422790527344, 1029.826171875]","[213.6416015625, 1029.886474609375]","[213.778564453125, 1029.9365234375]","[213.89016723632812, 1029.9779052734375]","[213.98062133789062, 1030.0118408203125]","[214.05349731445312, 1030.039794921875]","[214.11175537109375, 1030.0623779296875]","[214.15786743164062, 1030.080810546875]","[214.19383239746094, 1030.0955810546875]","[214.22137451171875, 1030.107421875]"
34150,"[730.2833251953125, 1518.2611083984375]","[732.9799194335938, 1523.7767333984375]","[733.0866088867188, 1521.0760498046875]","[733.6114501953125, 1520.2767333984375]","[733.9990844726562, 1519.1165771484375]","[734.3895874023438, 1518.1153564453125]","[734.7471313476562, 1517.151611328125]","[735.0761108398438, 1516.246826171875]","[735.3763427734375, 1515.3980712890625]","[735.6495361328125, 1514.605712890625]",...,"[737.479248046875, 1508.89990234375]","[737.58642578125, 1508.5802001953125]","[737.689697265625, 1508.2828369140625]","[737.78955078125, 1508.005859375]","[737.8865966796875, 1507.747314453125]","[737.981201171875,

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
26725,410.643372,1154.229980,411.249268,1159.430664,412.508148,1161.099854,413.122345,1162.585815,413.615021,1163.890503,...,417.195740,1184.564819,417.341705,1185.512085,417.495575,1186.449585,417.655334,1187.371216,417.817383,1188.267700
10945,1729.334351,465.846100,1732.179077,468.022400,1729.851929,470.934509,1729.052246,471.772888,1728.319214,472.521484,...,1724.229248,481.786224,1724.312012,482.353546,1724.437256,482.988190,1724.636719,483.629639,1724.857422,484.313446
583,731.913147,1549.999634,732.034546,1545.675781,732.615051,1544.552124,733.092224,1543.130615,733.586853,1542.128540,...,737.431885,1528.763794,737.466492,1528.404297,737.492981,1528.060181,737.511780,1527.730713,737.523010,1527.415283
37070,189.343430,1025.128540,193.459198,1026.066162,196.359894,1026.427612,198.421143,1026.697266,200.270996,1026.857300,...,214.053497,1030.039795,214.111755,1030.062378,214.157867,1030.080811,214.193832,1030.095581,214.221375,1030.107422
34150,730.283325,1518.261108,732.979919,1523.776733,733.086609,1521.076050,733.611450,1520.276733,733.999084,1519.116577,...,737.981201,1507.505371,738.073792,1507.278687,738.164734,1507.065918,738.254517,1506.866211,738.343506,1506.678833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28315,1711.809082,492.234528,1715.153931,493.947449,1714.043213,496.568054,1714.860352,497.323303,1714.758545,498.214325,...,1713.676270,507.462250,1713.524658,507.673706,1713.373413,507.867462,1713.224609,508.046112,1713.079712,508.212067
2383,741.598145,1392.721069,738.341187,1393.000854,737.220947,1393.418945,735.974976,1393.276001,735.083374,1393.224121,...,730.961731,1388.712036,730.940979,1388.379517,730.923706,1388.037964,730.909424,1387.687744,730.897888,1387.329224
14934,1689.309448,330.468689,1688.970093,328.365204,1688.216797,327.888733,1687.388184,327.079285,1686.794189,326.780640,...,1681.833130,317.745209,1681.744995,317.298950,1681.657104,316.848145,1681.568848,316.393738,1681.479980,315.937195


In [12]:
df2.to_csv("outputs6epb4all_batchnormEncoderDecoder.csv", index=True)